In [18]:
using Pkg
Pkg.activate("/media/mat/HDD/EnKF/")

"/media/mat/HDD/EnKF/Project.toml"

In [19]:
using Revise
using EnKF
using Distributions
using LinearAlgebra

In [41]:
@time α = MvNormal([1.0],[0.1]) 

# @time rand(α, 10)

  0.000005 seconds (10 allocations: 608 bytes)


DiagNormal(
dim: 1
μ: [1.0]
Σ: [0.01]
)


In [21]:
typeof(α)

MvNormal{Float64,PDMats.PDiagMat{Float64,Array{Float64,1}},Array{Float64,1}}

In [22]:
typeof(α) <: MultivariateDistribution

true

In [23]:
length(α)

1

In [24]:
size(α)

(1,)

In [42]:
A = MvNormal(zeros(10), 1.05*I)

IsoNormal(
dim: 10
μ: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Σ: [1.1025 0.0 … 0.0 0.0; 0.0 1.1025 … 0.0 0.0; … ; 0.0 0.0 … 1.1025 0.0; 0.0 0.0 … 0.0 1.1025]
)


In [40]:
1.05^2

1.1025

In [26]:
std(rand(A))

0.9899532415522562

In [27]:
diag([2.0 1.0 3.0])

1-element Array{Float64,1}:
 2.0

In [28]:
size(T)

(10,)

In [29]:
var(T)

10-element Array{Float64,1}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [30]:
T = AdditiveInflation(10)

AdditiveInflation{10}(IsoNormal(
dim: 10
μ: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Σ: [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0]
)
)

In [31]:
size(T)

(10,)

In [32]:
length(T)

10

In [33]:
M = MultiplicativeInflation(10)

MultiplicativeInflation{10}(1.0)

In [35]:
MultiplicativeInflation(1.01)

MethodError: MethodError: no method matching MultiplicativeInflation(::Float64)
Closest candidates are:
  MultiplicativeInflation(!Matched::Int64) at /media/mat/HDD/EnKF/src/inflation.jl:104
  MultiplicativeInflation(!Matched::Int64, !Matched::Real) at /media/mat/HDD/EnKF/src/inflation.jl:108

In [36]:
M = MultiAdditiveInflation(10)

MultiAdditiveInflation{10}(1.0, IsoNormal(
dim: 10
μ: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Σ: [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0]
)
)

In [43]:
M = MultiAdditiveInflation(10, 1.05, A)

MultiAdditiveInflation{10}(1.05, IsoNormal(
dim: 10
μ: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Σ: [1.1025 0.0 … 0.0 0.0; 0.0 1.1025 … 0.0 0.0; … ; 0.0 0.0 … 1.1025 0.0; 0.0 0.0 … 0.0 1.1025]
)
)